<a href="https://colab.research.google.com/github/yukinaga/ai_stock_prediction/blob/main/section_3/02_rnn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIによる株価予測（LSTM版）（データの取得・学習・推論）
ここではモデル定義クラス LSTMNet を作成し、時系列の順序を保ったまま学習を行います。

In [ ]:
# 必要なライブラリのインストールとインポート
# !pip install -q yfinance

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# --- 設定 ---
STOCK_CODE = '7203.T'  # 予測対象（例: トヨタ自動車）
START_DATE = '2020-01-01'
END_DATE = '2024-12-31'
WINDOW_SIZE = 15     # 過去30日分のデータから翌日を予測
TEST_RATIO = 0.2
EPOCHS = 200
BATCH_SIZE = 32
HIDDEN_SIZE = 64     # LSTMの隠れ層のニューロン数
NUM_LAYERS = 2       # LSTMの層の数

# 1. データの取得と前処理
print("データを取得中...")
df = yf.download(STOCK_CODE, start=START_DATE, end=END_DATE, progress=False)

# 株価の「変動率（リターン）」を計算
data = df['Close'].pct_change().dropna().values.reshape(-1, 1)

# データの正規化
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)

# データセットの作成関数
def create_dataset(dataset, window_size):
    X, y = [], []
    for i in range(len(dataset) - window_size):
        feature = dataset[i:i + window_size]
        target = dataset[i + window_size]
        X.append(feature)
        y.append(target)
    return np.array(X), np.array(y)

X, y = create_dataset(data_normalized, WINDOW_SIZE)

# 学習用とテスト用に分割
split_index = int(len(X) * (1 - TEST_RATIO))
X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

# PyTorchのTensorに変換
# LSTMへの入力形状は (Batch_Size, Sequence_Length, Input_Size) です
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

# データローダーの作成
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# 2. LSTMモデルの定義
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size=1):
        super(LSTMNet, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # batch_first=True にすると (batch, seq, feature) で入出力できる
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # x: (batch_size, window_size, input_size)
        # 初期隠れ状態とセル状態を初期化（デバイス合わせ）
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # LSTMへ通す
        out, _ = self.lstm(x, (h0, c0))

        # out[:, -1, :] : 最後のタイムステップの出力のみを取り出す
        # 多対一 (Many-to-One) の予測のため
        out = self.fc(out[:, -1, :])
        return out

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 入力サイズは1（変動率という1つの特徴量のみ）
model = LSTMNet(input_size=1, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS).to(device)

# 損失関数と最適化手法
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 3. 学習ループ
print("LSTMモデルの学習を開始します...")
train_losses = []

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{EPOCHS}, Loss: {epoch_loss:.6f}')

print("学習完了。")

plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss (LSTM)')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()

# 予測結果の評価 その1（視覚化と誤差の計算）

In [ ]:
# 評価モードへ
model.eval()

# 推論の実行
with torch.no_grad():
    predictions = model(X_test_tensor.to(device)).cpu().numpy()

# 正規化を元に戻す
predictions_actual = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test)

# グラフによる可視化（直近100日分）
plt.figure(figsize=(14, 6))
plt.plot(y_test_actual[-100:], label='Actual Return', color='blue', alpha=0.6)
plt.plot(predictions_actual[-100:], label='Predicted Return (LSTM)', color='green', alpha=0.7)

plt.title(f'{STOCK_CODE} Return Prediction with LSTM (Last 100 Days)')
plt.xlabel('Time (Days)')
plt.ylabel('Return Rate')
plt.legend()
plt.grid(True)
plt.show()

# 誤差の計算
rmse = np.sqrt(np.mean((predictions_actual - y_test_actual)**2))
print(f"Test RMSE (LSTM): {rmse:.6f}")

# 予測結果の評価 その2（投資シミュレーション）

In [ ]:
# 投資シミュレーション
initial_capital = 100000.0

# LSTM戦略
lstm_capital = [initial_capital]
current_lstm_capital = initial_capital

# ガチホ戦略
hold_capital = [initial_capital]
current_hold_capital = initial_capital

for i in range(len(y_test_actual)):
    actual_return = y_test_actual[i][0]
    predicted_return = predictions_actual[i][0]

    # LSTMが「上がる」と予測した場合のみ投資
    if predicted_return > 0:
        current_lstm_capital = current_lstm_capital * (1 + actual_return)

    lstm_capital.append(current_lstm_capital)

    # ガチホは常に投資
    current_hold_capital = current_hold_capital * (1 + actual_return)
    hold_capital.append(current_hold_capital)

# 結果の可視化
plt.figure(figsize=(14, 6))
plt.plot(lstm_capital, label='AI Strategy (LSTM)', color='green')
plt.plot(hold_capital, label='Buy & Hold Strategy', color='blue', linestyle='--')

plt.title('Investment Simulation: LSTM vs Buy & Hold')
plt.xlabel('Days')
plt.ylabel('Portfolio Value')
plt.legend()
plt.grid(True)
plt.show()

print(f"最終資産 (Buy & Hold): {hold_capital[-1]:.2f}")
print(f"最終資産 (AI Strategy - LSTM): {lstm_capital[-1]:.2f}")

if lstm_capital[-1] > hold_capital[-1]:
    print("結果: LSTMモデルが市場平均を上回りました。")
else:
    print("結果: LSTMモデルは市場平均に及びませんでした。")